### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

#[-]File to Load
#jupyter does not recognize __file__ adding this to avoid
#rewriting the code if it gets exported
try:
    __thisdir__ = os.path.dirname(os.path.abspath(__file__))
except NameError as error:
    __thisdir__ = "."

__csv_path__ = os.path.join(__thisdir__,"Resources","purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(__csv_path__)

## Player Count

* Display the total number of players


In [2]:
total_players  = len(purchase_data["SN"].unique())
print(f"Total Players: {total_players}")

Total Players: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
#Get the purchasing information and stored in variables
unique_items = len(purchase_data["Item ID"].unique())
average_price = "$"+str(round(purchase_data["Price"].mean(),2))
number_of_purchases = len(purchase_data)
total_rev = "$"+str(purchase_data["Price"].sum())
#Create the data frame
purchase_df = pd.DataFrame({"Number of unique items": pd.Series(unique_items,index = ["Value"]),
                            "Average price"         : pd.Series(average_price,index = ["Value"]),
                            "Number of purchases"   : pd.Series(number_of_purchases,index = ["Value"]),
                            "Total revenue"         : pd.Series(total_rev,index = ["Value"])})
purchase_df

,Number of unique items,Average price,Number of purchases,Total revenue
Value,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#Create a new frame with the desired information
gen_dem_df = pd.DataFrame(purchase_data[["Gender","SN"]])
#Remove duplicated users
#Note: This can be a bug if the user enters a different gender an exeption is added
#after this to protect the code against that scenario
gen_dem_df.drop_duplicates(inplace=True)

#To avoid issues where a user added multiple gender, compare the total players
#vs the data frame that associates the gender and user name
if total_players != len(gen_dem_df):
    msg = "There is at least 1 user with multiple genders, please review your data-set"
    raise RuntimeError(msg)

#Get the total gender values
gen_total = gen_dem_df["Gender"].value_counts()
#Calculate the percentage
gen_perc = round((gen_total/total_players)*100,2)
gen_perc = gen_perc.map('{:.2f}%'.format)
#Create a new data frame with the previos series
pd.DataFrame({"Total Count":gen_total,"Percentage of Players":gen_perc})

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#Create a new frame to associate gender and price only
gen_price_df = pd.DataFrame(purchase_data[["Gender","Price"]])

#Create a data frame with the total value spend per SN
sn_total_df = pd.DataFrame(purchase_data[["Price","SN"]])
sn_total_df = sn_total_df.groupby("SN").sum()

#[-] Purchase count
purchase_count_df = gen_price_df.groupby("Gender").count()

#[-] Average purchase price
avg_purchase_df = round(gen_price_df.groupby("Gender").mean(),2)

#[-] Total purchase value
total_purchase_df = gen_price_df.groupby("Gender").sum()

#[-] Avg Total Purchase per Person
#Merge the data with the depurated gender SN frame
merged_frame = pd.merge(sn_total_df,gen_dem_df, on="SN")
#Get the series with the average total purchase per person
avg_total_df = round(merged_frame.groupby("Gender").mean(),2)


purch_analysys_gen = \
            pd.DataFrame({"Purchase Count": purchase_count_df["Price"],
                        "Average Purchase Price": avg_purchase_df["Price"].map('${:.2f}'.format),
                        "Total Purchase Value": total_purchase_df["Price"].map('${:.2f}'.format),
                        "Avg Total Purchase per Person": avg_total_df["Price"].map('${:.2f}'.format)})
purch_analysys_gen

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#Filter data by age, Validate the data is the same for Age and SN duplicated same as above
age_demograph_df = pd.DataFrame(purchase_data[["SN","Age"]])
age_demograph_df.drop_duplicates(inplace=True)

#To avoid issues where a user added multiple ages, compare the total players
#vs the data frame that associates the age and user name
if total_players != len(age_demograph_df):
    msg = "There is at least 1 user with multiple ages, please review your data-set"
    raise RuntimeError(msg)
    
#Create the bins 
data_bin  = [ 0, 9, 14, 19, 24, 29, 34, 39, purchase_data["Age"].max()]
age_names = ["<10", "10-14","15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Cutting the frame
age_demograph_df["Age"] = pd.cut(age_demograph_df["Age"],data_bin,labels=age_names)
age_demograph_df.rename(columns={"Age":"Age Range"},inplace=True)

age_demo_res_df = age_demograph_df.groupby("Age Range").count()
age_demo_res_df.rename(columns={"SN":"Total Count"},inplace=True)

#Performing the total count
total_age_range = age_demo_res_df["Total Count"].sum()

#Calculating the percentage
age_demo_res_df["Percentage of Players"] = \
            round((age_demo_res_df["Total Count"]/total_age_range)*100,2).map('{:.2f}%'.format)

age_demo_res_df

,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#Data frame used to get SN and their price
sn_price_df = pd.DataFrame(purchase_data[["SN","Price"]])
#Merge the SN filtered gender data with the price
age_purch_analysis_df = pd.merge(age_demograph_df,sn_price_df, on="SN")

#[-] Purchase Count
purchase_count_ser = age_purch_analysis_df["Age Range"].value_counts()

#[-] Average Purchase Price
avg_purchase_price_df = age_purch_analysis_df.groupby("Age Range").mean()
avg_purchase_price_df["Price"] = round(avg_purchase_price_df["Price"],2)

#[-] Total purchase
total_purchase_df = age_purch_analysis_df.groupby("Age Range").sum()

#[-] Total purchase
avg_tot_purch_per_df = age_purch_analysis_df.groupby("SN")

#[-] Avg Total Purchase per Person
avg_tot_x_person_df = pd.merge(sn_total_df, age_demograph_df, on="SN")
avg_tot_x_person_df = round(avg_tot_x_person_df.groupby("Age Range").mean(),2)

purch_analysys_age = \
    pd.DataFrame({"Purchase Count": purchase_count_ser,
                "Average Purchase Price": avg_purchase_price_df["Price"].map('${:.2f}'.format),
                "Total Purchase Value": total_purchase_df["Price"].map('${:.2f}'.format),
                "Avg Total Purchase per Person": avg_tot_x_person_df["Price"].map('${:.2f}'.format)})
purch_analysys_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#Data frame used to process the top spenders
top_spenders_slice_df = pd.DataFrame(purchase_data[["SN","Price"]])
#[-] Purchase Count
pur_count_df = top_spenders_slice_df.groupby("SN").count()
#[-] Total Purchase Value
total_val_df = top_spenders_slice_df.groupby("SN").sum()
#[-] Average Purchase Price
avg_purch_val = round(total_val_df["Price"]/pur_count_df["Price"],2)

#Creating the data frame
top_spenders_df = pd.DataFrame({"Purchase Count": pur_count_df["Price"],
                                "Average Purchase Price": avg_purch_val,
                                "Total Purchase Value": total_val_df["Price"]})
#Sort by Total Purchase Value
top_spenders_df.sort_values(by=["Total Purchase Value"], ascending=False, inplace=True)

#Format teh table
top_spenders_df["Average Purchase Price"] = top_spenders_df["Average Purchase Price"].map('${:.2f}'.format)
top_spenders_df["Total Purchase Value"] = top_spenders_df["Total Purchase Value"].map('${:.2f}'.format)

#Show preview
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
#Data frame used to process the top spenders
items_slice_df = pd.DataFrame(purchase_data[["Item Name","Price"]])

#[-] Purchase Count
item_count_df = items_slice_df.groupby("Item Name").count()

#[-] Total Purchase Value
item_total_val_df = items_slice_df.groupby("Item Name").sum()

#[-] Total Purchase Value
#Adding this code to notify the users that there is at least 1 item with
#different prices. This scenario is posible based on nature of the data-set
if len(items_slice_df.drop_duplicates()) != len(item_count_df):
    #Create the message to display
    msg = "[Warning]:There is at least 1 item with multiple prices\n"
    msg += "any price information will be averaged"
    #Calculate the lines for message
    if len(msg.split("\n")) >=1:
        line_size = len(msg.split("\n")[0])
    else:
        line_size = len(msg)
    #Creating the line
    line = "-"*line_size
    #Printing the message
    print( line + "\n" + msg + "\n" + line )

#By calculating the mean we avoid issues where a single item has multiple prices
item_price_df = round(items_slice_df.groupby("Item Name").mean(),2)

#Creating the data frame
item_popular_df = pd.DataFrame({"Purchase Count": item_count_df["Price"],
                                "Item Price": item_price_df["Price"],
                                "Total Purchase Value": item_total_val_df["Price"]})
#Sort by Purchase Count
item_popular_df.sort_values(by=["Purchase Count"], ascending=False, inplace=True)

#Formating the table
pplr_fmt_df = item_popular_df.copy()

pplr_fmt_df["Item Price"] = item_popular_df["Item Price"].map('${:.2f}'.format)
pplr_fmt_df["Total Purchase Value"] = item_popular_df["Total Purchase Value"].map('${:.2f}'.format)

#Show preview
pplr_fmt_df.head()

-------------------------------------------------------
[Warning]:There is at least 1 item with multiple prices
any price information will be averaged
-------------------------------------------------------


,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,13,$4.61,$59.99
"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
Persuasion,9,$3.22,$28.99
Nirvana,9,$4.90,$44.10
"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
#Sort by Purchase count
item_popular_df.sort_values(by=["Total Purchase Value"], ascending=False, inplace=True)

#Formating the table
profit_fmt_df = item_popular_df.copy()

profit_fmt_df["Item Price"] = item_popular_df["Item Price"].map('${:.2f}'.format)
profit_fmt_df["Total Purchase Value"] = item_popular_df["Total Purchase Value"].map('${:.2f}'.format)

#Show preview
profit_fmt_df.head()

,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,13,$4.61,$59.99
"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
Nirvana,9,$4.90,$44.10
Fiery Glass Crusader,9,$4.58,$41.22
Singed Scalpel,8,$4.35,$34.80
